In [1]:
import sys
# assert Python 3.7 or later is required
assert sys.version_info >= (3, 7)

import cv2 as cv
import numpy as np

In [2]:
pip install imutils


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=1574557139d121d21e2a2b972f014d87aac0319989ea9a64e8e28458889a717d
  Stored in directory: c:\users\grace\appdata\local\pip\cache\wheels\31\d0\2c\87ce38f6052879e5b7b18f0f8b4a10ad2a9d210e908d449f16
Successfully built imutils


# Exercise 1
Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix.

In [5]:
# using imutils
import imutils

img = cv.imread('images/lena.jfif')

# Rotate the image by 45 degrees without cropping
rotated_img = imutils.rotate_bound(img, 45)

# Display the original and rotated images
cv.imshow("Original Image", img)
cv.imshow("Rotated Image", rotated_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [29]:
# Modify the transformation matrix

img = cv.imread('images/lena.jfif')

# Get the center and the rotation matrix
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
angle = 45
scale = 1.0

# Modify the transformation matrix to avoid cropping
M = cv.getRotationMatrix2D(center, angle, scale)

# Get the sine and cosine of the angle
sin_theta = abs(M[0, 1])
cos_theta = abs(M[0, 0])

# Compute the new bounding dimensions of the image
new_w = int((h * sin_theta) + (w * cos_theta))
new_h = int((h * cos_theta) + (w * sin_theta))

# Adjust the rotation matrix to take the new dimensions into account
M[0, 2] += (new_w / 2) - center[0]
M[1, 2] += (new_h / 2) - center[1]

# Apply the transformation to the image
rotated_img = cv.warpAffine(img, M, (new_w, new_h))

# Display the original and rotated images
cv.imshow("Original Image", img)
cv.imshow("Rotated Image", rotated_img)
cv.waitKey(0)
cv.destroyAllWindows()

# Exercise 2
Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower. Please refer to online documentation for more info. 

In [28]:
img1 = cv.imread('images/flower.jfif')
img2 = cv.imread('images/native-bee.png')

assert img1 is not None, "file could not be read, check with os.path.exists()"
assert img2 is not None, "file could not be read, check with os.path.exists()"

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img1.shape
roi = img2[0:rows, 0:cols]

# Now create a mask of logo and create its inverse mask also
img1gray = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# Now black-out the area of logo in ROI
img2_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img1_fg = cv.bitwise_and(img1,img1,mask = mask)

# Put logo in ROI and modify the main image
dst = cv.add(img2_bg,img1_fg)
img2[0:rows, 0:cols ] = dst
cv.imshow('res',img2)
cv.waitKey(0)
cv.destroyAllWindows()

# Exercise 3
Apply custom sharpening kernel of aperture size 3 and 5 as shown below on 'native-bee.png':

In [27]:
# Load the 'native-bee.png' image
image = cv.imread('images/native-bee.png')

# Define custom sharpening kernels of aperture size 3 and 5
kernel_size_3 = np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]], dtype=np.float32)

kernel_size_5 = np.array([[-1, -1, -1, -1, -1],
                          [-1, -1, -1, -1, -1],
                          [-1, -1, 25, -1, -1],
                          [-1, -1, -1, -1, -1],
                          [-1, -1, -1, -1, -1]], dtype=np.float32)

# Apply the custom sharpening kernels using filter2D function
sharpened_image_3 = cv.filter2D(image, -1, kernel_size_3)
sharpened_image_5 = cv.filter2D(image, -1, kernel_size_5)

# Display the original and sharpened images
cv.imshow("Sharpened Image (Aperture Size 3)", sharpened_image_3)
cv.imshow("Sharpened Image (Aperture Size 5)", sharpened_image_5)
cv.waitKey(0)
cv.destroyAllWindows()

### What can you infer from the outputs?
The sharpened image with an aperture size of 3 shows moderate edge enhancement, while the sharpened image with an aperture size of 5 exhibits a more pronounced sharpening effect with stronger edge enhancement and details.

# Exercise 4
Apply different image smoothing techniques (e.g. average filter, Gaussian kernel and median filter) on 'noise_lena.jpg' and display the resulting images after the convolution. Comment on the outcomes and deduce the type of noise present on the image.

In [26]:
# Load the 'noise_lena.jpg' image
image = cv.imread('images/noise_lena.jpg')

# Apply average filter (box filter)
average_filtered = cv.blur(image, (5, 5))

# Apply Gaussian filter
gaussian_filtered = cv.GaussianBlur(image, (5, 5), 0)

# Apply median filter
median_filtered = cv.medianBlur(image, 5)

# Display the original and filtered images
cv.imshow("Average Filtered Image", average_filtered)
cv.imshow("Gaussian Filtered Image", gaussian_filtered)
cv.imshow("Median Filtered Image", median_filtered)
cv.waitKey(0)
cv.destroyAllWindows()

### Comment on the outcomes and deduce the type of noise present on the image.
**1. Average filter**
- appears slightly blurred compared to the original image. It reduces high-frequency noise but can also blur edges and fine details.
- Gaussian noise

**2. Gaussian filter**
- smoother than the original, but it preserves edges better than the average filter. It reduces noise while maintaining image details.
- Gaussian noise

**3. Median filter**
- effectively removes salt-and-pepper noise from the image while preserving edges. It tends to perform better than the other filters in removing impulse noise.
- impulse noise (i.e., salt-and-pepper noise)